### 상장회사 자료 들고오기

In [6]:
import requests
from bs4 import BeautifulSoup as bs

url = 'https://finance.naver.com/sise/sise_deposit.naver'
html = requests.get(url)
soup = bs(html.content)

data = soup.select_one('div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

import re

target_day = re.findall('[0-9]+', data)
target_day = ''.join(target_day)

from io import BytesIO
import pandas as pd

gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_stk = {
    'locale': 'ko_KR',
    'mktId': 'STK',
    'trdDd': target_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020506'}
otp_stk = requests.post(gen_url, gen_otp_stk, headers=headers).text
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_html_stk = requests.post(down_url, {'code': otp_stk}, headers=headers)
sector_stk = pd.read_csv(BytesIO(down_html_stk.content), encoding='EUC-KR')

gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_ksq = {
    'locale': 'ko_KR',
    'mktId': 'KSQ',
    'trdDd': target_day,
    'money': '1',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020506'}
otp_ksq = requests.post(gen_url, gen_otp_ksq, headers=headers).text
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_html_ksq = requests.post(down_url, {'code': otp_ksq}, headers=headers)
sector_ksq = pd.read_csv(BytesIO(down_html_ksq.content), encoding='EUC-KR')

krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop = True)
krx_sector['종목명'] = krx_sector['종목명'].str.strip()
krx_sector['기준일'] = target_day

In [7]:
krx_sector

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일
0,095570,AJ네트웍스,KOSPI,서비스업,4475,15,0.34,209529770125,20230517
1,006840,AK홀딩스,KOSPI,기타금융,20650,350,1.72,273562134650,20230517
2,027410,BGF,KOSPI,기타금융,4095,-10,-0.24,391960259145,20230517
3,282330,BGF리테일,KOSPI,유통업,180100,-2100,-1.15,3112831470600,20230517
4,138930,BNK금융지주,KOSPI,기타금융,6670,20,0.30,2173988090820,20230517
...,...,...,...,...,...,...,...,...,...
2580,024060,흥구석유,KOSDAQ,유통,5540,-10,-0.18,83100000000,20230517
2581,010240,흥국,KOSDAQ,기계·장비,6620,210,3.28,81576247520,20230517
2582,189980,흥국에프엔비,KOSDAQ,음식료·담배,2840,45,1.61,113991428680,20230517
2583,037440,희림,KOSDAQ,기타서비스,9120,0,0.00,126972972000,20230517


### 파생상품 종목 들고오기

In [15]:
import requests
from bs4 import BeautifulSoup as bs
from io import BytesIO
import pandas as pd

url = 'https://finance.naver.com/sise/sise_deposit.naver'
html = requests.get(url)
soup = bs(html.content)

data = soup.select_one('div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

import re

target_day = re.findall('[0-9]+', data)
target_day = ''.join(target_day)

gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_drv = {    	
    'locale': 'ko_KR',
    'prodId': 'KRDRVFUK2I',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT12801'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020506'}
otp_drv = requests.post(gen_url, gen_otp_drv, headers=headers).text
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_html_drv = requests.post(down_url, {'code': otp_drv}, headers=headers)
list_drv = pd.read_csv(BytesIO(down_html_drv.content), encoding='EUC-KR')

In [16]:
list_drv

,표준코드,단축코드,한글종목명,한글종목약명,영문종목명,상장일,최종거래일,최종결제일,기초자산유형,거래승수
0,KR4101T60006,101T6000,코스피200 F 202306,F 202306,KOSPI 200 F 202306,2021/06/11,2023/06/08,2023/06/09,지수(Index),250000
1,KR4101T90003,101T9000,코스피200 F 202309,F 202309,KOSPI 200 F 202309,2022/09/13,2023/09/14,2023/09/15,지수(Index),250000
2,KR4101TC0008,101TC000,코스피200 F 202312,F 202312,KOSPI 200 F 202312,2020/12/11,2023/12/14,2023/12/15,지수(Index),250000
3,KR4101V30005,101V3000,코스피200 F 202403,F 202403,KOSPI 200 F 202403,2023/03/10,2024/03/14,2024/03/15,지수(Index),250000
4,KR4101V60002,101V6000,코스피200 F 202406,F 202406,KOSPI 200 F 202406,2022/06/10,2024/06/13,2024/06/14,지수(Index),250000
5,KR4101VC0004,101VC000,코스피200 F 202412,F 202412,KOSPI 200 F 202412,2021/12/10,2024/12/12,2024/12/13,지수(Index),250000
6,KR4101WC0003,101WC000,코스피200 F 202512,F 202512,KOSPI 200 F 202512,2022/12/09,2025/12/11,2025/12/12,지수(Index),250000
7,KR4401T6T9S9,401T6T9S,코스피200 SP 2306-2309,SP 2306-2309,KOSPI 200 SP 2306-2309,2023/03/10,2023/06/08,2023/06/09,지수(Index),250000
8,KR4401T6TCS1,401T6TCS,코스피200 SP 2306-2312,SP 2306-2312,KOSPI 200 SP 2306-2312,2023/03/10,2023/06/08,2023/06/09,지수(Index),250000
9,KR4401T6V3S8,401T6V3S,코스피200 SP 2306-2403,SP 2306-2403,KOSPI 200 SP 2306-2403,2023/03/10,2023/06/08,2023/06/09,지수(Index),250000


### 파생상품 가격 가져오기

In [19]:
import requests
from bs4 import BeautifulSoup as bs
from io import BytesIO
import pandas as pd

url = 'https://finance.naver.com/sise/sise_deposit.naver'
html = requests.get(url)
soup = bs(html.content)

data = soup.select_one('div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

import re

target_day = re.findall('[0-9]+', data)
target_day = ''.join(target_day)

gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_drv_price = {
    'locale': 'ko_KR',
    'trdDd': target_day,
    'prodId': 'KRDRVFUK2I',
    'trdDdBox1': '20230519',
    'trdDdBox2': '20230519',
    'mktTpCd': 'T',
    'rghtTpCd': 'T',
    'share': '1',
    'money': '3',
    'csvxls_isNo': 'false',
    'name': 'fileDown',
    'url': 'dbms/MDC/STAT/standard/MDCSTAT12501'
}
headers = {'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020506'}
otp_drv_price = requests.post(gen_url, gen_otp_drv_price, headers=headers).text
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_html_drv = requests.post(down_url, {'code': otp_drv_price}, headers=headers)
list_drv_price = pd.read_csv(BytesIO(down_html_drv.content), encoding='EUC-KR')

In [20]:
list_drv_price

,종목코드,종목명,종가,대비,시가,고가,저가,현물가,정산가,거래량,거래대금,미결제약정
0,101T6000,코스피200 F 202306 (주간),326.50,0.95,325.45,327.95,325.10,326.61,326.50,160867,13141695.0,318578.0
1,101T9000,코스피200 F 202309 (주간),328.00,0.90,326.85,329.30,326.60,326.61,328.00,1232,101106.0,27747.0
2,101TC000,코스피200 F 202312 (주간),329.15,-0.85,329.15,329.15,329.15,326.61,329.15,201,16570.0,26842.0
3,101V3000,코스피200 F 202403 (주간),327.00,-3.00,327.00,327.00,327.00,326.61,327.00,201,16537.0,8979.0
4,101V6000,코스피200 F 202406 (주간),NaN,NaN,NaN,NaN,NaN,326.61,331.25,0,0.0,8090.0
5,101VC000,코스피200 F 202412 (주간),NaN,NaN,NaN,NaN,NaN,326.61,335.15,0,0.0,11580.0
6,101WC000,코스피200 F 202512 (주간),NaN,NaN,NaN,NaN,NaN,326.61,339.30,0,0.0,15.0
7,401T6T9S,코스피200 SP 2306-2309 (주간),1.45,1.45,1.40,1.45,1.40,326.61,NaN,1064,174553.0,NaN
8,401T6TCS,코스피200 SP 2306-2312 (주간),NaN,NaN,NaN,NaN,NaN,326.61,NaN,0,0.0,NaN
9,401T6V3S,코스피200 SP 2306-2403 (주간),2.10,2.10,2.15,2.15,2.10,326.61,NaN,20,3283.0,NaN
